# Importing libraries

In [1]:
import pandas as pd

import psycopg
from psycopg import sql

import os
from dotenv import load_dotenv, find_dotenv

# First look at the data

In [2]:
try:
    load_dotenv()
    password = os.getenv('db_password')

    with psycopg.connect(f"dbname=Final-project user=postgres password={password}") as conn:
        cursor = conn.cursor()
        query = "SELECT * FROM car_sales;"
        cursor.execute(query)

        rows = cursor.fetchall()

    columns = [desc[0] for desc in cursor.description]
    sales_df = pd.DataFrame(rows, columns=columns)

    sales_df.head() 

except : 
    sales_df = pd.read_csv('../Datasets/car_price_dataset.csv')

In [3]:
col_names_initial = list(sales_df.columns)
col_names = {
    col_names_initial[0] : 'ID',
    col_names_initial[1] : 'price',
    col_names_initial[2] : 'levy',
    col_names_initial[3] : 'manufacturer',
    col_names_initial[4] : 'model',
    col_names_initial[5] : 'production_year',
    col_names_initial[6] : 'category',
    col_names_initial[7] : 'lether_interior',
    col_names_initial[8] : 'fuel_type',
    col_names_initial[9] : 'engine_size',
    col_names_initial[10] : 'mileage',
    col_names_initial[11] : 'cylinders',
    col_names_initial[12] : 'gearbox_type',
    col_names_initial[13] : 'drive_type',
    col_names_initial[14] : 'doors',
    col_names_initial[15] : 'wheel',
    col_names_initial[16] : 'colors',
    col_names_initial[17] : 'airbags'
}
sales_df.rename(columns = col_names, inplace = True)
sales_df.columns

Index(['ID', 'price', 'levy', 'manufacturer', 'model', 'production_year',
       'category', 'lether_interior', 'fuel_type', 'engine_size', 'mileage',
       'cylinders', 'gearbox_type', 'drive_type', 'doors', 'wheel', 'colors',
       'airbags'],
      dtype='object')

In [4]:
sales_df.head()

,ID,price,levy,manufacturer,model,production_year,category,lether_interior,fuel_type,engine_size,mileage,cylinders,gearbox_type,drive_wheels,doors,wheel,color,airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


# First exploration

In [5]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               19237 non-null  int64  
 1   price            19237 non-null  int64  
 2   levy             19237 non-null  object 
 3   manufacturer     19237 non-null  object 
 4   model            19237 non-null  object 
 5   production_year  19237 non-null  int64  
 6   category         19237 non-null  object 
 7   lether_interior  19237 non-null  object 
 8   fuel_type        19237 non-null  object 
 9   engine_size      19237 non-null  object 
 10  mileage          19237 non-null  object 
 11  cylinders        19237 non-null  float64
 12  gearbox_type     19237 non-null  object 
 13  drive_wheels     19237 non-null  object 
 14  doors            19237 non-null  object 
 15  wheel            19237 non-null  object 
 16  color            19237 non-null  object 
 17  airbags     

In [6]:
sales_df.duplicated().value_counts()

False    18924
True       313
dtype: int64

In [7]:
sales_df.drop_duplicates(inplace=True, ignore_index=True)

In [8]:
sales_df.duplicated().value_counts()

False    18924
dtype: int64

In [9]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18924 entries, 0 to 18923
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               18924 non-null  int64  
 1   price            18924 non-null  int64  
 2   levy             18924 non-null  object 
 3   manufacturer     18924 non-null  object 
 4   model            18924 non-null  object 
 5   production_year  18924 non-null  int64  
 6   category         18924 non-null  object 
 7   lether_interior  18924 non-null  object 
 8   fuel_type        18924 non-null  object 
 9   engine_size      18924 non-null  object 
 10  mileage          18924 non-null  object 
 11  cylinders        18924 non-null  float64
 12  gearbox_type     18924 non-null  object 
 13  drive_wheels     18924 non-null  object 
 14  doors            18924 non-null  object 
 15  wheel            18924 non-null  object 
 16  color            18924 non-null  object 
 17  airbags     

In [10]:
sales_df = sales_df.rename({'ID': 'index'}, axis=1).set_index('index')

In [11]:
sales_df.head()

,price,levy,manufacturer,model,production_year,category,lether_interior,fuel_type,engine_size,mileage,cylinders,gearbox_type,drive_wheels,doors,wheel,color,airbags
index,,,,,,,,,,,,,,,,,
45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


# Columns cleaning and formating

## levy

In [12]:
sales_df['levy'].value_counts()

-       5709
765      482
891      453
639      403
640      398
        ... 
3156       1
2908       1
1279       1
1719       1
1901       1
Name: levy, Length: 559, dtype: int64

In [13]:
def str_to_int_levy(x) :
    if x == '-':
        x = 0
    else :
        x = int(x)
    return x

In [14]:
sales_df['levy'] = sales_df['levy'].apply(lambda x : str_to_int_levy(x))

In [15]:
sales_df['levy'].value_counts()

0       5709
765      482
891      453
639      403
640      398
        ... 
3156       1
2908       1
1279       1
1719       1
1901       1
Name: levy, Length: 559, dtype: int64

## manufacturer

In [16]:
liste = list(sales_df['manufacturer'].unique())

In [17]:
sorted(liste)

['ACURA',
 'ALFA ROMEO',
 'ASTON MARTIN',
 'AUDI',
 'BENTLEY',
 'BMW',
 'BUICK',
 'CADILLAC',
 'CHEVROLET',
 'CHRYSLER',
 'CITROEN',
 'DAEWOO',
 'DAIHATSU',
 'DODGE',
 'FERRARI',
 'FIAT',
 'FORD',
 'GAZ',
 'GMC',
 'GREATWALL',
 'HAVAL',
 'HONDA',
 'HUMMER',
 'HYUNDAI',
 'INFINITI',
 'ISUZU',
 'JAGUAR',
 'JEEP',
 'KIA',
 'LAMBORGHINI',
 'LANCIA',
 'LAND ROVER',
 'LEXUS',
 'LINCOLN',
 'MASERATI',
 'MAZDA',
 'MERCEDES-BENZ',
 'MERCURY',
 'MINI',
 'MITSUBISHI',
 'MOSKVICH',
 'NISSAN',
 'OPEL',
 'PEUGEOT',
 'PONTIAC',
 'PORSCHE',
 'RENAULT',
 'ROLLS-ROYCE',
 'ROVER',
 'SAAB',
 'SATURN',
 'SCION',
 'SEAT',
 'SKODA',
 'SSANGYONG',
 'SUBARU',
 'SUZUKI',
 'TESLA',
 'TOYOTA',
 'UAZ',
 'VAZ',
 'VOLKSWAGEN',
 'VOLVO',
 'ZAZ',
 'სხვა']

## category

In [18]:
liste = list(sales_df['category'].unique())
sorted(liste)

['Cabriolet',
 'Coupe',
 'Goods wagon',
 'Hatchback',
 'Jeep',
 'Limousine',
 'Microbus',
 'Minivan',
 'Pickup',
 'Sedan',
 'Universal']

## engine_size

In [19]:
liste = list(sales_df['engine_size'].unique())
sorted(liste)

['0',
 '0.1',
 '0.2',
 '0.2 Turbo',
 '0.3',
 '0.3 Turbo',
 '0.4',
 '0.4 Turbo',
 '0.5',
 '0.6',
 '0.6 Turbo',
 '0.7',
 '0.7 Turbo',
 '0.8',
 '0.8 Turbo',
 '0.9',
 '1',
 '1.0 Turbo',
 '1.1',
 '1.1 Turbo',
 '1.2',
 '1.2 Turbo',
 '1.3',
 '1.3 Turbo',
 '1.4',
 '1.4 Turbo',
 '1.5',
 '1.5 Turbo',
 '1.6',
 '1.6 Turbo',
 '1.7',
 '1.7 Turbo',
 '1.8',
 '1.8 Turbo',
 '1.9',
 '1.9 Turbo',
 '2',
 '2.0 Turbo',
 '2.1',
 '2.1 Turbo',
 '2.2',
 '2.2 Turbo',
 '2.3',
 '2.3 Turbo',
 '2.4',
 '2.4 Turbo',
 '2.5',
 '2.5 Turbo',
 '2.6',
 '2.7',
 '2.7 Turbo',
 '2.8',
 '2.8 Turbo',
 '2.9',
 '2.9 Turbo',
 '20',
 '3',
 '3.0 Turbo',
 '3.1',
 '3.2',
 '3.2 Turbo',
 '3.3',
 '3.4',
 '3.5',
 '3.5 Turbo',
 '3.6',
 '3.6 Turbo',
 '3.7',
 '3.7 Turbo',
 '3.8',
 '3.9',
 '4',
 '4.0 Turbo',
 '4.2',
 '4.2 Turbo',
 '4.3',
 '4.4',
 '4.4 Turbo',
 '4.5',
 '4.5 Turbo',
 '4.6',
 '4.6 Turbo',
 '4.7',
 '4.7 Turbo',
 '4.8',
 '4.8 Turbo',
 '5',
 '5.0 Turbo',
 '5.2',
 '5.3',
 '5.4',
 '5.4 Turbo',
 '5.5',
 '5.5 Turbo',
 '5.6',
 '5.7',
 '5.7

In [20]:
def get_turbo(x):
    try:
        x.split(' ')[1]
        return 1
    except:
        return 0

In [21]:
sales_df['turbo'] = sales_df['engine_size'].apply(lambda x : get_turbo(x))
sales_df['engine_size'] = sales_df['engine_size'].apply(lambda x : float((x.split(' '))[0]))
sales_df.head()

,price,levy,manufacturer,model,production_year,category,lether_interior,fuel_type,engine_size,mileage,cylinders,gearbox_type,drive_wheels,doors,wheel,color,airbags,turbo
index,,,,,,,,,,,,,,,,,,
45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12,0
44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8,0
45774419,8467,0,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2,0
45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0,0
45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4,0


In [22]:
sales_df['turbo'].value_counts()

0    17032
1     1892
Name: turbo, dtype: int64

In [23]:
sales_df['engine_size'].dtype

dtype('float64')

## mileage

In [24]:
sales_df['mileage'] = sales_df['mileage'].apply(lambda x : int((x.split(' '))[0]))
sales_df.head()

,price,levy,manufacturer,model,production_year,category,lether_interior,fuel_type,engine_size,mileage,cylinders,gearbox_type,drive_wheels,doors,wheel,color,airbags,turbo
index,,,,,,,,,,,,,,,,,,
45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,04-May,Left wheel,Silver,12,0
44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8,0
45774419,8467,0,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,04-May,Right-hand drive,Black,2,0
45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,04-May,Left wheel,White,0,0
45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,04-May,Left wheel,Silver,4,0


## cylinders

In [25]:
sales_df['cylinders'] = sales_df['cylinders'].apply(lambda x : int(x))

## doors

In [26]:
doors_dict = {
    '02-Mar' : 2,
    '04-May' : 4,
    '>5' : 5
}
    
sales_df['doors'].replace(doors_dict, inplace = True)

In [27]:
sales_df.head()

,price,levy,manufacturer,model,production_year,category,lether_interior,fuel_type,engine_size,mileage,cylinders,gearbox_type,drive_wheels,doors,wheel,color,airbags,turbo
index,,,,,,,,,,,,,,,,,,
45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6,Automatic,4x4,4,Left wheel,Silver,12,0
44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000,6,Tiptronic,4x4,4,Left wheel,Black,8,0
45774419,8467,0,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4,Variator,Front,4,Right-hand drive,Black,2,0
45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4,Automatic,4x4,4,Left wheel,White,0,0
45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4,Automatic,Front,4,Left wheel,Silver,4,0


In [28]:
liste = list(sales_df['airbags'].unique())
sorted(liste)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [29]:
sales_df.head()

,price,levy,manufacturer,model,production_year,category,lether_interior,fuel_type,engine_size,mileage,cylinders,gearbox_type,drive_wheels,doors,wheel,color,airbags,turbo
index,,,,,,,,,,,,,,,,,,
45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6,Automatic,4x4,4,Left wheel,Silver,12,0
44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000,6,Tiptronic,4x4,4,Left wheel,Black,8,0
45774419,8467,0,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4,Variator,Front,4,Right-hand drive,Black,2,0
45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4,Automatic,4x4,4,Left wheel,White,0,0
45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4,Automatic,Front,4,Left wheel,Silver,4,0


In [30]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18924 entries, 45654403 to 45813273
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            18924 non-null  int64  
 1   levy             18924 non-null  int64  
 2   manufacturer     18924 non-null  object 
 3   model            18924 non-null  object 
 4   production_year  18924 non-null  int64  
 5   category         18924 non-null  object 
 6   lether_interior  18924 non-null  object 
 7   fuel_type        18924 non-null  object 
 8   engine_size      18924 non-null  float64
 9   mileage          18924 non-null  int64  
 10  cylinders        18924 non-null  int64  
 11  gearbox_type     18924 non-null  object 
 12  drive_wheels     18924 non-null  object 
 13  doors            18924 non-null  int64  
 14  wheel            18924 non-null  object 
 15  color            18924 non-null  object 
 16  airbags          18924 non-null  int64  
 17  tu

In [88]:
#sales_df.to_csv('../Datasets/car_price_prediction-preprocess.csv', index=False) 

In [38]:
load_dotenv()
password = os.getenv('db_password')

cols = ",".join([str(i) for i in sales_df.columns.tolist()])
types = ['INT',
         'INT',
         'VARCHAR (100)',
         'VARCHAR (100)',
         'INT',
         'VARCHAR (100)',
         'VARCHAR (50)',
         'VARCHAR (50)',
         'FLOAT',
         'INT',
         'INT',
         'VARCHAR (50)',
         'VARCHAR (50)',
         'INT',
         'VARCHAR (50)',
         'VARCHAR (50)',
         'INT',
         'INT'
]

with psycopg.connect(f"dbname=Final-project user=postgres password={password}") as conn:
    cursor = conn.cursor()

    query = "CREATE TABLE car_sales_cleaned (" + ",".join([f"{col} {types[i]}" for i, col in enumerate(sales_df.columns)]) + ");"
    cursor.execute(query)

    for i, row in sales_df.iterrows():
        query = "INSERT INTO car_sales_cleaned (" + cols + ") VALUES (" + "%s," * (len(row) - 1) + "%s);"
        cursor.execute(query, tuple(row))